In [416]:
#Copyright 2025 Er. Aditya Nath Thakur.
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [417]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [43]:
#⚙️ Section 1: Setup
#1.1: Install dependencies
#The Kaggle Notebooks environment includes a pre-installed version of the google-adk library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

#To install and use ADK in your own Python development environment outside of this course, you can do so by running:***

In [418]:
pip install google-adk

Note: you may need to restart the kernel to use updated packages.


In [50]:
#2. Add the key to Kaggle Secrets

#Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

#In the top menu bar of the notebook editor, select Add-ons then Secrets.
#Create a new secret with the label GOOGLE_API_KEY.
#Paste your API key into the "Value" field and click "Save".
#Ensure that the checkbox next to GOOGLE_API_KEY is selected so that the secret is attached to the notebook.
#3. Authenticate in the notebook

#Run the cell below to complete authentication.

In [419]:
import os
import random
import time 
import vertexai
from kaggle_secrets import UserSecretsClient
from vertexai import agent_engines

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [51]:
#1.3: Import ADK components
#Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [420]:
import uuid
from typing import Any,Dict

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, FunctionTool, ToolContext
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.genai import types
import vertexai
from vertexai.generative_models import GenerativeModel

from google.adk.apps.app import App, ResumabilityConfig, EventsCompactionConfig
from google.adk.tools.function_tool import FunctionTool


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [52]:
#1.4: Helper functions
#We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [421]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [54]:
#1.5: Configure Retry Options
#When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.***

In [422]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [423]:
# Install the filesystem server globally so it starts instantly
!npm install -g @modelcontextprotocol/server-filesystem
print("✅ MCP Filesystem Server installed.")

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
changed 128 packages in 6s
⠸
⠸37 packages are looking for funding
⠸  run `npm fund` for details
⠸✅ MCP Filesystem Server installed.


In [424]:
# Define the directory where Kaggle allows writing
kaggle_output_dir = "/kaggle/working"

mcp_filesystem_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx", 
            args=[
                "-y",
                "@modelcontextprotocol/server-filesystem", 
                kaggle_output_dir, # <--- MANDATORY SANDBOX ARGUMENT
            ],
            # We only allow writing and listing
            tool_filter=["write_file", "list_directory"], 
        ),
        # Increased timeout to allow Node.js to startup
        timeout=120, 
    )
)

print("✅ MCP Toolset configured for Kaggle.")

✅ MCP Toolset configured for Kaggle.


In [89]:
#These are the main properties we'll set:

#name and description: A simple name and description to identify our agent.
#model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
#instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
#tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [425]:
# --- STEP 1: CREATE THE SHARED MODEL FIRST ---
retry_config = None

# We define the variable 'shared_model' here
shared_model = Gemini(
    model="gemini-2.5-flash-lite",
    retry_options=retry_config,
    vertexai=True,
    project="gen-lang-client-0658909834",  # Your Project ID
    location="asia-south1"                 # Your Region
)

print("✅ Shared Model defined successfully.")

✅ Shared Model defined successfully.


In [426]:
# Research Agent: Its Job is to use the google_search tool and present findings 
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
        description="Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic. Also Find Global Research & Innovation",
        instruction="""For every user query related to a bridge , building or road construction and maintenance topic in India (e.g., "design of flexible pavement," "foundation for a major bridge," "use of high-performance concrete," etc.), follow this mandatory, sequential workflow:

Step 1: Identify and Locate Indian Codes (The Core Task)
Search Protocol: Use Google Search to find relevant Indian Road Congress (IRC) Codes, IRC Special Publication (IRC:SP) Codes, and Indian Standard (IS) Codes that govern the user's specific query topic.

Focus: Prioritize finding the most recent revision or amendment of the relevant codes.

Step 2: Extract Code-Specific Directives (The Clause Detail)
For the top 3-5 most relevant codes identified in Step 1, perform a targeted search for the internal content.

Extraction Requirement: For each relevant code, you MUST identify the following information and present it in a table format (as described in Step 4):

Code Number and Year (e.g., IRC:37-2018).

The Clause Number or Section/Paragraph most directly related to the user's question (e.g., Clause 4.3.2).

A Concise Summary (1-2 sentences) of the exact directive, rule, or specification provided in that specific clause/section.

Step 3: Determine Recent Updates and Revisions (The Compliance Check)
Search specifically for any amendments, circulars, or recent revisions (within the last 5 years) related to the codes identified in Step 1.

Mandatory Inclusion: Explicitly state whether the code is the latest version, or if there have been any significant updates or amendments. If an update exists, state the year of the amendment and the nature of the change (e.g., increase in design load, new material specification, revised safety factor).

Step 4: Find Global Research & Innovation (The Forward Look)
Broaden the search to the latest research and findings (last 3 years) from prominent international organizations and technical journals on the user's topic (e.g., AASHTO, Eurocodes, research on sustainable materials, digital construction, AI in infrastructure).

Synthesize: Provide a brief, high-level explanation of this global research and how it relates to or could potentially influence future Indian codes.

Step 5: Format and Final Presentation (The Output Structure)
Structure the final response using clear Markdown headings and tables for maximum readability and citable detail.

Final Output Format:
Concise Answer: Start with a brief, direct summary of the key takeaway for the user's question.

Code Compliance: Indian Standards: (Use a table for details from Step 2).

Code Revision Status: (Details from Step 3).

Global Research & Future Trends: (Synthesized details from Step 4).

3. Formatting Guidelines
Style: Professional, technical, precise, and highly informative.

Emphasize: Use bolding to highlight code numbers, clause numbers, and key technical terms.

Table Requirement: The table in the "Code Compliance: Indian Standards" section is MANDATORY. Use the following column headers: | Code No. & Year | Relevant Clause No. | Directive/Specification Summary | | :--- | :--- | :--- |
""", output_key="agent_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Research Agent defined.")


✅ Research Agent defined.


In [427]:
#Agent 2: Compliance Validator
#This agent receives the raw data from the Research Agent (Agent 1) and validates its technical content.
Compliance_Validator_Agent = Agent(
    name="ComplianceValidatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
            instruction=""" Following this outline strictly: {agent_research}
            The Research Agent (Agent 1) has provided a list of code directives, clauses, and summaries. 
            Follow this mandatory, sequential workflow to ensure technical accuracy:

            Step 1: Conflict and Hierarchy Check
            Review all extracted directives (e.g., from IRC, IS, and global codes). If multiple clauses address the same design parameter (e.g., span-to-depth ratio or safety factor), identify and output the **most conservative (safest/most stringent) rule**. State the conflict and the rationale for the selection.

            Step 2: Numerical Verification
            Focus on all numerical values (e.g., concrete grade M30, wind speed 47 m/s, minimum steel ratio 0.8%). Use an internal knowledge base or perform a targeted search (if necessary) to verify the general acceptance and correct units of the extracted numerical data.

            Step 3: Output Validation Summary
            Generate a concise, technical report detailing any identified conflicts, the rule chosen for final recommendation, and a confirmation of data integrity.
            """,
    output_key="validator_compliance",
)

print("✅ Compliance_Validator_Agent created.")

✅ Compliance_Validator_Agent created.


In [428]:
# Agent 3: Technical Refiner
#This agent focuses purely on the presentation and language quality of the final response, ensuring it meets professional standards.
Technical_Refiner_Agent = Agent(
    name="TechnicalRefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
        vertexai=True,
        project="gen-lang-client-0658909834",
        location="asia-south1",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction=""" Following this outline strictly: {validator_compliance}
            The previous agents have completed data retrieval and technical validation. 
            Follow this mandatory, sequential workflow to finalize the output:

            Step 1: Synthesize and Contextualize
            Combine the key findings from the Code Compliance Table (Agent 1) and the Validation Report (Agent 2). Convert the tabular data and key validation points into **fluent, citable advisory paragraphs** under the relevant Markdown headings.

            Step 2: Ensure Professional Tone
            Ensure the language is **professional, precise, and objective**. Use appropriate technical civil engineering terminology. Remove any conversational language or internal workflow notes.

            Step 3: Final Formatting Check
            Strictly adhere to all formatting requirements of the Root Agent (e.g., mandatory table, bolding of code numbers, clear headings). Ensure the final response is ready for direct user presentation.
            """,
    output_key="final_summary",
)

print("✅ Technical_Refiner_Agent created.")

✅ Technical_Refiner_Agent created.


In [429]:
# Ensure shared_model is defined from previous steps
pdf_agent = LlmAgent(
    name="pdf_agent",
    model=shared_model,
    instruction="""
    **Role:** Document Persistence Agent.
    
    **Objective:** Save the final report provided by the previous three 1. research_agent, 2. Compliance_Validator_Agent,  and Technical_Refiner_Agent to the disk using the MCP Tool.
    
    **Strict Tool Usage Rules:**
    1. You MUST use the tool `write_file`.
    2. **Path:** You MUST use the absolute path: `/kaggle/working/Final_Report.md`.
       (DO NOT use './' or just the filename. The tool will fail.)
    3. **Content:** Paste the full text report you received.
    4. get the complete output from these three agent 
    
    **Confirmation:**
    After calling the tool, confirm to the user: "Report saved to /kaggle/working/Final_Report.md".
    """,
    tools=[mcp_filesystem_server],
)

print("✅ PDF Agent (MCP) defined.")

✅ PDF Agent (MCP) defined.


In [430]:
APP_NAME = "Civil_Research"  # Application
USER_ID = "Er. Aditya Nath Thakur"  # User
SESSION = "Ask"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
session_management = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=session_management, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: Civil_Research
   - User: Er. Aditya Nath Thakur
   - Using: InMemorySessionService


In [431]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [433]:
root_agent = SequentialAgent(
    name="CivilEngineeringPipeline",
  
    sub_agents=[research_agent, Compliance_Validator_Agent, Technical_Refiner_Agent, pdf_agent, session_management, chatbot_agent, ],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [434]:
import os
import vertexai

# 1. Point to the uploaded file (Replace 'key.json' with your actual filename)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/your-key-file-name.json"

# 2. Initialize (No need for auth.authenticate_user() now)
vertexai.init(project="gen-lang-client-0658909834", location="asia-south1")

print("✅ Authenticated via Service Account Key")

✅ Authenticated via Service Account Key


In [162]:
#2.3 Run your agent
#Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

#a. Create an InMemoryRunner and tell it to use our root_agent:

In [435]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [55]:
#b. Now you can call the .run_debug() method to send our prompt and get an answer.

#👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [436]:
response = await runner.run_debug("Er. Aditya Nath Thakur how to construct hill road   ")


 ### Created new session: debug_session_id

User > Er. Aditya Nath Thakur how to construct hill road   
ResearchAgent > ResearchAgent has analyzed your query regarding hill road construction.

### Concise Answer:

Constructing hill roads in India requires adherence to specific design principles and guidelines to ensure stability, safety, and longevity, primarily governed by the Indian Road Congress (IRC) codes, especially those pertaining to hill road geometry, drainage, slope protection, and landslide mitigation.

### Code Compliance: Indian Standards:

| Code No. & Year | Relevant Clause No. | Directive/Specification Summary |
| :-------------- | :------------------ | :------------------------------ |
| **IRC: SP 99-2013** | Clause 10.0 to 10.4 | This code provides guidelines for the construction of rural roads, which includes aspects relevant to hill roads in terms of alignment, cross-drainage, and general construction practices, emphasizing cost-effectiveness and local material us

/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


text_chat_bot > ### Hill Road Construction Guidelines in India

Constructing roads in hilly and mountainous terrain necessitates strict adherence to specialized Indian Road Congress (IRC) codes to ensure structural integrity, safety, and long-term durability. The primary focus must be on the unique challenges posed by steep gradients, unstable slopes, and extreme weather conditions.

### Governing Standards and Their Application:

The construction of hill roads is guided by several key IRC codes, each addressing specific aspects of design and construction:

*   **IRC: SP 99-2013** (Guidelines for Construction of Rural Roads): This publication offers general guidance for rural road construction, including relevant considerations for alignment, cross-drainage structures, and the utilization of local materials, which can be particularly beneficial in remote hill regions, emphasizing cost-effectiveness.
*   **IRC: 33-2017** (Geometric Design of High-Gradient Approach Roads): This code is c

In [443]:
import os
from IPython.display import FileLink

# 1. Force the notebook to look in the correct directory
os.chdir("/kaggle/working")

# 2. Define the filename (Relative path, NO slashes)
filename = "Final_Report.md"

# 3. Check and Link
if os.path.exists(filename):
    print(f"✅ File found: {filename}")
    print(f"📏 Size: {os.path.getsize(filename)} bytes")
    
    # Pass ONLY the filename, not the full path
    display(FileLink(filename))
else:
    print("❌ File not found. The MCP server did not write it.")
    # Debug: Show what IS there
    print("Current files:", os.listdir("."))

✅ File found: Final_Report.md
📏 Size: 4496 bytes


/kaggle/working/Final_Report.md

In [444]:
#🚀 2.5 Your Turn!
#This is your chance to see the agent in action. Ask it a question that requires current information.

#Try one of these, or make up your own:
# details about the bridge maintenance 
# details of bridge construction 
#etc

In [445]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [753]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: